In [1]:
%%capture 
!pip install trl

### **Implementing GRPO in TRL**

---

#### **GRPO là gì?**
GRPO (Group Relative Policy Optimization) là một thuật toán tăng cường đặc biệt trong dòng RLHF, giúp mô hình học **từ sự so sánh giữa các nhóm phản hồi (completions)** – thay vì đánh giá từng phản hồi riêng lẻ.

---

### **Các khái niệm chính trong GRPO**

1. **Group Formation**:  
   Mô hình sinh ra **nhiều completions cho mỗi prompt**, tạo thành một nhóm để so sánh.

2. **Preference Learning**:  
   Một **hàm reward** được định nghĩa để **so sánh và xếp hạng các phản hồi** trong nhóm → từ đó điều chỉnh policy.

3. **Training Configuration**:  
   Việc huấn luyện được điều khiển thông qua `GRPOConfig`.

---

### **Để triển khai GRPO với TRL, cần:**

| Bước | Mô tả |
|------|------|
| 1️⃣ | Định nghĩa dataset chứa **prompt** |
| 2️⃣ | Định nghĩa **reward function**: đầu vào là list completions, đầu ra là list điểm reward |
| 3️⃣ | Tạo `GRPOConfig` để cấu hình quá trình huấn luyện |
| 4️⃣ | Sử dụng `GRPOTrainer` để huấn luyện mô hình |

In [ ]:
from trl import GRPOTrainer, GRPOConfig
from datasets import load_dataset

# 1. Load your dataset
dataset = load_dataset("your_dataset", split="train")


# 2. Define a simple reward function
def reward_func(completions, **kwargs):
    """Example: Reward longer completions"""
    return [float(len(completion)) for completion in completions]


# 3. Configure training
training_args = GRPOConfig(
    output_dir="output",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    logging_steps=10,
)

# 4. Initialize and train
trainer = GRPOTrainer(
    model="your_model",  # e.g. "Qwen/Qwen2-0.5B-Instruct"
    args=training_args,
    train_dataset=dataset,
    reward_funcs=reward_func,
)
trainer.train()

### **Key Components**

## **1. Dataset Format**

Dataset cần phải chứa **prompt** mà mô hình sẽ tạo ra câu trả lời. GRPO Trainer sẽ tạo ra nhiều câu trả lời cho mỗi **prompt** và sử dụng **reward function** để so sánh chúng.

## **2. Reward Function**

Reward function định hướng việc học. Bạn sẽ cần viết một hàm nhận vào một list completions và trả ra list điểm số (reward).



In [2]:
# Example 1: Reward based on completion length
def reward_length(completions, **kwargs):
    return [float(len(completion)) for completion in completions]


# Example 2: Reward based on matching a pattern
import re

def reward_format(completions, **kwargs):
    pattern = r"^<think>.*?</think><answer>.*?</answer>$"
    return [1.0 if re.match(pattern, c) else 0.0 for c in completions]

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## **3. `GRPOConfig`**

| Tham số | Ý nghĩa | Gợi ý |
|---------|--------|--------|
| `num_generation` | Số completions được sinh ra/mỗi prompt | **4–8** cho bài toán nhẹ, **8–16** nếu bài toán phức tạp (như NL → FOL reasoning) |
| `use_vllm` | Dùng backend VLLM để tăng tốc sinh text | Bật nếu bạn dùng GPU lớn hoặc inference nhiều |
| `per_device_train_batch_size` | Nên bằng `num_generation` nếu có thể | Đảm bảo mỗi batch chứa đủ nhóm phản hồi |


In [3]:
training_args = GRPOConfig(
    # Essential parameters
    output_dir="output",
    num_train_epochs=3,
    num_generation=4,  # Number of completions to generate for each prompt
    per_device_train_batch_size=4,  # We want to get all generations in one device batch
    # Optional but useful
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    logging_steps=10,
    # GRPO specific (optional)
    use_vllm=True,  # Speed up generation
)

NameError: name 'GRPOConfig' is not defined

### **Tips for Success**

1. **Quản lý bộ nhớ:** Điều chỉnh `per_device_train_batch_size` và `gradient_accumulation_steps` tùy theo dung lượng GPU.
2. **Tăng tốc:** Bật `use_vllm=True` nếu mô hình của bạn được hỗ trợ, để tăng tốc độ sinh văn bản.
3. **Giám sát:** Theo dõi các chỉ số trong quá trình huấn luyện:
  - `reward`: Phần thưởng trung bình của các câu sinh ra.
  - `reward_std`: Độ lệch chuẩn của phần thưởng giữa các nhóm.
  - `kl`: Độ lệch KL so với mô hình tham chiếu.

### **Reward Function Design**
1. **Length-Based Rewards**: Một trong những reward dễ thiết kế nhất là dựa trên độ dài. Có thể thiết kế để phạt các mà phản hồi quá ngắn hoặc quá dài.
2. **Rule-Based Rewards for Verifiable Tasks**: Đối với những nhiệm vụ có câu trả lời đúng khách quan (như mã hóa hoặc tính toán), có thể triển khai phần thưởng dựa trên quy tắc (ví dụ, nếu trả lời đúng +1, sai +0).
3. **Format-Based Rewards**: Có thể thiết kế phần thưởng dựa trên format của câu trả lời (ví dụ, nếu phản hồi có <think> token và thỏa mãn +1).